In [95]:
# common
import os
BASE_DIR = os.path.dirname(os.path.realpath('__file__'))

from datetime import datetime, timedelta
from io import BytesIO
from pathlib import Path

# network & protocol
import requests
from requests.exceptions import BaseHTTPError

# computational
import numpy as np
import pandas as pd

In [142]:
stations = [
    {
        'name': 'Neak_Luong',
        'stage_id': '0c55bc1fb44e4caf881226bbf8a6e62a',
    },
    {
        'name': 'Stung_Treng',
        'discharge_id': ['2066a06f57764afca57a6be0f3eff8a7', 'dcd4b92a9d9341859f5f1658836166f1'],
    },
    {
        'name': 'Chaktomuk',
        'stage_id': ['7f84c4d851a543d1a5eccc53947e0d3c', '2c070f6a900b41e7ad19bafef374f52d'],
        'seasonal': True,
    },
    {
        'name': 'Kompong_Kdei',
        'stage_id': '75ba98758e7c4eb3b105f733db150307',
    },
    {
        'name': 'Prek_Kdam',
        'discharge_id': '6615982ab17b4a38a5269e5ba4ef8a4f',
        'stage_id': ['81e259eb15f64135834fe9e388ee7fbb', '5a7037f272c745d5bcafcc16a5c2f89c'],
        'seasonal': True,
    },
    {
        'name': 'Kg._Thmar',
        'stage_id': '549c7f63d6dc4392b233b2049bd23d11',
    },
    {
        'name': 'Koh_Khel',
        'stage_id': '64758ae8f24a46759d962ab9a2dea27a',
    },
    {
        'name': 'Lumphat',
        'discharge_id': 'f17adb5f893a4f40924f698978906b87',
        'stage_id': ['93337ffe7838467289bf5b55401ee6f8', '6955b9585cc94524bcff184034e4afd3']
    },
    {
        'name': 'Kompong_Cham',
        'stage_id': 'b5da800e9c8e4346a3b8d9d6daa73e0a',
    },
    {
        'name': 'Phnom_Penh_Port',
        'stage_id': 'cd94408bcf944bc88033cf05675034fd',
    },
    {
        'name': 'Voeun_Sai',
        'discharge_id': 'eacd08c3a27e4fc5a3aa1af35792f6e3',
        'stage_id': ['d177b0f2dae547ceb9bc7c23ef2970ae', 'e821b15baedd4af2a4ecde624b3bc976']
    },
    {
        'name': 'Kratie',
        'discharge_id': ['17c0ffe74e224b50a9e3142fc404262b', 'ecc50c5540564a27bb9f7f147027bd9f'],
    },
]

In [143]:
count = (datetime.strptime(end_date, '%Y-%m-%d') - datetime.strptime(start_date, '%Y-%m-%d')).days + 1
count

2922

In [144]:
start_date = '2013-01-01'
end_date = '2020-12-31'
date_tail = 'T00:00:00.000Z'
request_param = dict(sd=f'{start_date}{date_tail}', ed=f'{end_date}{date_tail}')
full_dates = pd.date_range(datetime.strptime(start_date, '%Y-%m-%d'),
                           datetime.strptime(end_date, '%Y-%m-%d'), freq='d')\
             .strftime('%Y-%m-%d').tolist()

In [145]:
# some needed variables
base_url = 'https://api.mrcmekong.org/api/v1/ts/highcharts'
# request_param = dict(sd=sd, ed=ed)
headers = {
    'x-mrc-data-portal-chart-key': '56b6f095-2243-4d73-9bcf-57600ef1f38b',
    'Cache-Control': 'no-cache',
    'Pragma': 'no-cache',
    'User-Agent': 'PostmanRuntime/7.26.10',
    'Connection': 'keep-alive',
    'Accept': '*/*',
}

In [146]:
sess = requests.Session()

In [147]:
def get_data(url, request_param, headers, column):
    res = sess.get(url, params=request_param, headers=headers)
    if res.status_code == 200:
        response = json.loads(res.content)
        observations = response['series'][0]['data']
        df = pd.DataFrame(observations, columns=['datetime', column])
        df.datetime = pd.to_datetime(df.datetime, unit='ms', origin='unix')
#         df.datetime = df.datetime.apply(lambda d: datetime.datetime.utcfromtimestamp(d/1000.0).strftime('%Y-%m-%d %H:%M:%S'))
        df[df[column] < 0] = np.nan
        df = df.dropna()
        return df
    else:
        print(f'server returned status code {res.status_code} for {column} id')

In [148]:
def open_load_json(file_path):
    f = open(file_path)
    parameters = json.load(f)
    f.close()
    return parameters['m'], parameters['c']

In [149]:
Path(f'{BASE_DIR}/mrc_observations').mkdir(parents=True, exist_ok=True)

for station in stations:    
    name = station['name']
    print('-------------------------------------------')
    print(f'station: {name}')

    cols = ['datetime', 'discharge']
    df_discharge = pd.DataFrame(columns = cols)
    
    if 'discharge_id' in station:
        discharge_id = station['discharge_id']
        if isinstance(discharge_id, list):
            for _id in discharge_id:
                _df = get_data(f'{base_url}/{_id}', request_param, headers, cols[1])
                _df = _df.groupby(_df.datetime.dt.date).mean()
                _df['datetime'] = _df.index
                if not df_discharge.empty and not _df.empty:
                    # replace measured with calculated
                    df_discharge.loc[df_discharge.datetime.isin(_df.datetime), cols] = _df[cols]
                else:
                    df_discharge = _df
        else:
            df_discharge = get_data(f'{base_url}/{discharge_id}', request_param, headers, cols[1])
            df_discharge = df_discharge.groupby(df_discharge.datetime.dt.date).mean()
            df_discharge['datetime'] = df_discharge.index

    if len(df_discharge) == count:
        df_discharge.to_csv(f'{BASE_DIR}/mrc_observations/{name}.csv', encoding='utf-8', index=False)

    if len(df_discharge) > 0:
        dates = df_discharge.datetime.tolist()
        dates = [date.strftime('%Y-%m-%d') for date in dates]
    else:
        dates = []

    print(f'discharge: len(dates): {len(dates)}')

    if 'stage_id' in station:
        stage_id = station['stage_id']

        cols = ['datetime', 'stage']
        df_stage = pd.DataFrame(columns = cols)

        if isinstance(stage_id, list):
            for _id in stage_id:
                _df = get_data(f'{base_url}/{_id}', request_param, headers, cols[1])
                _df = _df.groupby(_df.datetime.dt.date).mean()
                _df['datetime'] = _df.index
                if not df_stage.empty and not _df.empty:
                    df_stage.loc[df_stage.datetime.isin(_df.datetime), cols] = _df[cols]
                else:
                    df_stage = _df
        else:
            df_stage = get_data(f'{base_url}/{stage_id}', request_param, headers, cols[1])
            df_stage = df_stage.groupby(df_stage.datetime.dt.date).mean()
            df_stage['datetime'] = df_stage.index
        
        print(f'before len(df_stage): {len(df_stage)}')

        remaining_dates = [datetime.strptime(_date, '%Y-%m-%d').date() for _date in full_dates if _date not in dates]
        df_stage['datetime'] = pd.to_datetime(df_stage['datetime'])
        df_stage = df_stage.loc[(df_stage['datetime'].dt.date.isin(remaining_dates)), :]
        print(f'after len(df_stage): {len(df_stage)}')

        # lets get the rating curves parameter
        if 'seasonal' in station:
            dry_m, dry_c = open_load_json(f'{BASE_DIR}/rating_curves/{name}_Seasonal/{name}_Seasonal_Dry_parameters.json')
            wet_m, wet_c = open_load_json(f'{BASE_DIR}/rating_curves/{name}_Seasonal/{name}_Seasonal_Wet_parameters.json')

            dry_df = df_stage[((df_stage['datetime'].dt.month >= 1) & (df_stage['datetime'].dt.month <= 4)) |
                              ((df_stage['datetime'].dt.month >= 11) & (df_stage['datetime'].dt.month <= 12))].copy()
            wet_df = df_stage[(df_stage['datetime'].dt.month >= 5) & (df_stage['datetime'].dt.month <= 10)].copy()
            
            # from manning's equation: Q = 10 ^ c * stage ^ m
            dry_df['discharge'] =  np.multiply(np.power(10, dry_c), np.power(dry_df.stage, dry_m))
            wet_df['discharge'] =  np.multiply(np.power(10, wet_c), np.power(wet_df.stage, wet_m))
            del dry_df['stage']
            del wet_df['stage']

            df = pd.concat([dry_df, wet_df, df_discharge])
            
        else:
            m, c = open_load_json(f'{BASE_DIR}/rating_curves/{name}/{name}_parameters.json')
            # from manning's equation: Q = 10 ^ c * stage ^ m
            df_stage['discharge'] =  np.multiply(np.power(10, c), np.power(df_stage['stage'], m))
            del df_stage['stage']

            df = pd.concat([df_stage, df_discharge])

        df.to_csv(f'{BASE_DIR}/mrc_observations/{name}.csv', encoding='utf-8', index=False)

-------------------------------------------
station: Neak_Luong
discharge: len(dates): 0
before len(df_stage): 2922
after len(df_stage): 2922
-------------------------------------------
station: Stung_Treng
discharge: len(dates): 2922
-------------------------------------------
station: Chaktomuk
discharge: len(dates): 0
before len(df_stage): 2922
after len(df_stage): 2922
-------------------------------------------
station: Kompong_Kdei
discharge: len(dates): 0
before len(df_stage): 2765
after len(df_stage): 2765
-------------------------------------------
station: Prek_Kdam
discharge: len(dates): 64
before len(df_stage): 2922
after len(df_stage): 2858
-------------------------------------------
station: Kg._Thmar
discharge: len(dates): 0
before len(df_stage): 2756
after len(df_stage): 2756
-------------------------------------------
station: Koh_Khel
discharge: len(dates): 0
before len(df_stage): 2922
after len(df_stage): 2922
-------------------------------------------
station: Lump